In [32]:
#Loading the Data sets
import pandas as pd
a = pd.read_csv("E:/DA_Projects/2/Customers.csv")
b = pd.read_csv("E:/DA_Projects/2/Products.csv")
c=pd.read_csv("E:/DA_Projects/2/Transactions.csv")

print(customers.head())
print(products.head())
print(transactions.head())


  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID TransactionDate  Quantity  TotalValue  \
0        T00001      C0199      P067      25-08-2024         1      300.68   
1        T00112      C0146      P067      27-05-2024         1      300.68   
2        T00166      C0127      P067      25-04-2024      

In [60]:
#Merging the Data 
merged_data = pd.merge(a, c, on='CustomerID')  
merged_data.to_csv("E:/DA_Projects/2/merged_data.csv", index=False)

In [61]:
#Merging the Data 
d="E:/DA_Projects/2/merged_data.csv"
merged_data = pd.merge(a, c, on='CustomerID')  
merged_data.to_csv("E:/DA_Projects/2/merged_data.csv", index=False)

In [34]:
d="E:/DA_Projects/2/merged_data.csv"

In [26]:
file_path = d
df = pd.read_csv(file_path)
print("Basic Info:")
print(df.info())  


Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    1000 non-null   object 
 1   CustomerID       1000 non-null   object 
 2   ProductID        1000 non-null   object 
 3   TransactionDate  1000 non-null   object 
 4   Quantity         1000 non-null   int64  
 5   TotalValue       1000 non-null   float64
 6   Price_x          1000 non-null   float64
 7   CustomerName     1000 non-null   object 
 8   Region           1000 non-null   object 
 9   SignupDate       1000 non-null   object 
 10  ProductName      1000 non-null   object 
 11  Category         1000 non-null   object 
 12  Price_y          1000 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 101.7+ KB
None


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import NearestNeighbors

file_path = d  
df = pd.read_csv(file_path)

In [23]:
#Creatin a KNN Model based on the Features given below by calculating the Euclidian distance
features = ["ProductID", "Quantity", "TotalValue", "Region"]
df_knn = df[["CustomerID"] + features].copy()  

label_encoders = {}
for col in ["ProductID", "Region"]:
    le = LabelEncoder()
    df_knn[col] = le.fit_transform(df_knn[col])
    label_encoders[col] = le 

scaler = StandardScaler()
df_knn[["Quantity", "TotalValue"]] = scaler.fit_transform(df_knn[["Quantity", "TotalValue"]])

knn = NearestNeighbors(n_neighbors=3, metric="euclidean")
knn.fit(df_knn.drop(columns=["CustomerID"])) 
def find_similar_customers(new_data, knn_model, label_encoders, scaler, df_original):
    for col in ["ProductID", "Region"]:
        if new_data[col] in label_encoders[col].classes_:
            new_data[col] = label_encoders[col].transform([new_data[col]])[0]
        else:
            print(f"Warning: {col} value '{new_data[col]}' not in training data. Using unknown encoding.")
            new_data[col] = -1  

    new_data_scaled = scaler.transform([[new_data["Quantity"], new_data["TotalValue"]]])

    input_vector = [[new_data["ProductID"], new_data_scaled[0][0], new_data_scaled[0][1], new_data["Region"]]]
    distances, indices = knn_model.kneighbors(input_vector)
    similar_customers = df_original.iloc[indices[0]][["CustomerID", "ProductID", "Quantity", "TotalValue", "Region"]].copy()
    similar_customers["Similarity Score"] = 1 / (1 + distances[0])  
    return similar_customers

In [24]:
#Testing the model on Custom Transaction Data And finding the similarity Score
product_id = input("Enter ProductID: ")
quantity = float(input("Enter Quantity: "))
total_value = float(input("Enter Total Value: "))
region = input("Enter Region: ")

new_customer = {
    "ProductID": product_id,
    "Quantity": quantity,
    "TotalValue": total_value,
    "Region": region
}
similar_customers = find_similar_customers(new_customer, knn, label_encoders, scaler, df_knn)
print("\nTop 3 Similar Customers:")
print(similar_customers)


Enter ProductID: PO62
Enter Quantity: 5
Enter Total Value: 1870
Enter Region: Asia

Top 3 Similar Customers:
    CustomerID  ProductID  Quantity  TotalValue  Region  Similarity Score
839      C0067          1  1.309264    1.410443       2          0.242399
645      C0024          0  1.309264   -0.025960       2          0.226539
651      C0017          0  0.414347   -0.369439       1          0.218182


C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [29]:
#Finding the similarity Model for a Given Set of Customers C0001 to C0020
features = ["ProductID", "Quantity", "TotalValue", "Region"]
df_knn = df[["CustomerID"] + features].copy()  # Keep CustomerID for reference
df_knn = df_knn[df_knn["CustomerID"].between("C0001", "C0020")].reset_index(drop=True)

label_encoders = {}
for col in ["ProductID", "Region"]:
    le = LabelEncoder()
    df_knn[col] = le.fit_transform(df_knn[col])
    label_encoders[col] = le  

scaler = StandardScaler()
df_knn[["Quantity", "TotalValue"]] = scaler.fit_transform(df_knn[["Quantity", "TotalValue"]])
knn = NearestNeighbors(n_neighbors=4, metric="euclidean")  
knn.fit(df_knn.drop(columns=["CustomerID"]))

lookalike_map = {}

for idx, row in df_knn.iterrows():
    customer_id = row["CustomerID"]
    customer_vector = row.drop("CustomerID").values.reshape(1, -1)
    distances, indices = knn.kneighbors(customer_vector)
    similar_customers = []
    for i in range(1, 4):  # Start from 1 to exclude self (0th index)
        similar_customer_id = df_knn.iloc[indices[0][i]]["CustomerID"]
        similarity_score = round(1 / (1 + distances[0][i]), 4)  # Convert distance to similarity score
        similar_customers.append((similar_customer_id, similarity_score))
    lookalike_map[customer_id] = similar_customers

lookalike_df = pd.DataFrame({"CustomerID": lookalike_map.keys(), "Lookalikes": [str(v) for v in lookalike_map.values()]})
lookalike_df["CustomerID_numeric"] = lookalike_df["CustomerID"].str.extract("(\d+)").astype(int)
lookalike_df = lookalike_df.sort_values(by="CustomerID_numeric").drop(columns="CustomerID_numeric")

# Save to file
output_file = "E:/DA_Projects/2/Lookalike.csv"
lookalike_df.to_csv(output_file, index=False)

print(f"Lookalike.csv saved successfully at: {output_file}")


Lookalike.csv saved successfully at: E:/DA_Projects/2/Lookalike.csv


C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\mayan\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not ha